## 중복구현

In [1]:
import pandas as pd
import numpy as np

path = '/content/drive/MyDrive/Colab Notebooks/final/'
web = pd.read_csv(path + 'dataset_genre.csv')

In [2]:
web.head()

title writer  ...                                              story platform
0      참교육    채용택  ...  무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용...   네이버 웹툰
1     신의 탑    SIU  ...     자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑   네이버 웹툰
2  뷰티풀 군바리     설이  ...                      여자도 군대에 간다면? 본격 여자도 군대 가는 만화!   네이버 웹툰
3   윈드브레이커    조용석  ...  혼자서 자전거를 즐겨타던 모범생 조자현. 원치 않게 자전거 크루의 일에 자꾸 휘말리...   네이버 웹툰
4     팔이피플     매미  ...  마스크걸 , 위대한 방옥숙 매미 희세 작가의 신작! 에서 육아용품 파는 평범한 유부...   네이버 웹툰

[5 rows x 7 columns]

In [3]:
matrics_path = '/content/drive/MyDrive/Colab Notebooks/final/matrics/'

okt_matrices = np.load(matrics_path+'okt_matrics_전처리후.npy',allow_pickle=True)
kkma_matrices = np.load(matrics_path+'kkma_matrics_전처리후.npy',allow_pickle=True)
hannanum_matrices = np.load(matrics_path+'hannanum_matrics_전처리후.npy',allow_pickle=True)
komoran_matrices = np.load(matrics_path+'komoran_matrics_전처리후.npy',allow_pickle=True)

In [4]:
from scipy.sparse import hstack

def hstack_matrices(matrices, noauthor, w_author, w_genre):
    cnt_genre_matrix, lb_writer_matrix, lb_illustrator_matrix, tfidf_story_matrix = matrices
    
    # tfidf_title_matrix,
    # 만들어진 행렬을 합치기 위한 리스트
    sparse_matrix_list = [cnt_genre_matrix * w_genre, tfidf_story_matrix]
    if not noauthor:
        sparse_matrix_list.append(lb_writer_matrix * w_author)
        sparse_matrix_list.append(lb_illustrator_matrix * w_author)
    
    # if not notitle:
    #     sparse_matrix_list.append(tfidf_title_matrix)
    
    sparse_matrix_list = tuple(sparse_matrix_list)

    # 행렬 결합
    X_features_sparse = hstack(sparse_matrix_list).tocsr()
#     print(type(X_features_sparse), X_features_sparse.shape)
    return X_features_sparse

In [5]:
# 웹툰 제목을 입력하면 해당 인덱스를 반환해줄 Series 생성
indices = pd.Series(web.index, index=web['title'])
indices.head(3)

title
참교육        0
신의 탑       1
뷰티풀 군바리    2
dtype: int64

In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tokenizer_matrices = [okt_matrices, kkma_matrices, hannanum_matrices, komoran_matrices]
# notitle=False,
def get_recomm(*title, rank=10, same_genre=False, noauthor=False,  w_author=0.7, w_genre=1.3, tokenizer_matrices=tokenizer_matrices):
    # 코사인 유사도를 담을 array 생성
    cosine_sim = np.zeros((len(web), len(web)))
    
    # 각 토큰화 도구로 만들어진 행렬을 합쳐서 코사인 유사도 생성
    for matrices in tokenizer_matrices:
        X_features_sparse = hstack_matrices(matrices, noauthor=noauthor,  w_author=w_author, w_genre=w_genre)
        cosine_sim += cosine_similarity(X_features_sparse, X_features_sparse)
    cosine_sim /= len(tokenizer_matrices)
    
    # 결과 출력을 위한 데이터 프레임 복사
    print_copy = web.copy()
    
    # 선택한 웹툰 제목으로부터 해당되는 인덱스를 받아옴
    idx = indices[title]
    if idx.size > 1:
        print(print_copy.iloc[idx,[1,2,4]])
        temp = int(input('원하는 작품을 선택하세요. : '))
        idx = idx[temp-1]
    else:
        pass

    # 인덱스로부터 장르를 뽑아옴
    genre = print_copy.loc[idx, 'genre']

    # 유사도를 DataFrame에 붙임
    print_copy['similarity'] = cosine_sim[idx]
    
    if same_genre:
        # 동일 장르만을 보여줄 경우
        return print_copy[print_copy['genre'] == genre].sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]
        
    else:
        # 다른 장르도 보여줄 경우
        return print_copy.sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]

In [ ]:
idx = indices['트리거']
type(idx)

pandas.core.series.Series

In [ ]:
idx.size

2

In [ ]:
idx

title
트리거      62
트리거    1513
dtype: int64

In [ ]:
web.iloc[62]

title                                                        트리거
writer                                                       고경빈
illustrator                                                  고경빈
day                                                            월
genre                                                        판타지
story          지구에 소행성이 떨어진 후, 지구는 인간을 먹이로 하는 괴생명체 아니스 에 의해 절...
platform                                                  네이버 웹툰
Name: 62, dtype: object

In [ ]:
get_recomm('신의 탑')

title writer  ... platform similarity
261   나 혼자 만렙 뉴비   메슬로우  ...   네이버 웹툰   0.482223
1441      창위의일루젼     시우  ...   네이버 웹툰   0.472798
89          빌런투킬     퓨핀  ...   네이버 웹툰   0.472261
931      환상적인 소년     후은  ...   네이버 웹툰   0.471816
3950   세상에 없는 이웃   1230  ...    레진코믹스   0.471722
715       골든 체인지    브림스  ...   네이버 웹툰   0.471299
60       오로지 오로라     홍달  ...   네이버 웹툰   0.470485
1179        붉은 실     동비  ...   네이버 웹툰   0.469633
1081       천년구미호     기량  ...   네이버 웹툰   0.469149
569   리안소울의 엑소클럽    신진오  ...   네이버 웹툰   0.469078

[10 rows x 8 columns]

In [ ]:
get_recomm('트리거')

     writer illustrator    genre
62      고경빈         고경빈      판타지
1513   불만있냥        불만있냥  학원, 판타지
원하는 작품을 선택하세요. : 1


title writer  ... platform similarity
468            마물전담부대     염손  ...   네이버 웹툰   0.473166
253            멸망X초이스      텐  ...   네이버 웹툰   0.471704
1118          하이브 1~2    김규삼  ...   네이버 웹툰   0.471233
819               아르마     병장  ...   네이버 웹툰   0.470957
251               카루나    강호진  ...   네이버 웹툰   0.470485
1319  2012 지구가 멸망한다면?   웹툰작가  ...   네이버 웹툰   0.470435
88              하우스키퍼    채용택  ...   네이버 웹툰   0.470305
1277            카오스어택    맛스타  ...   네이버 웹툰   0.469630
725              요괴대전    강두식  ...   네이버 웹툰   0.468905
89               빌런투킬     퓨핀  ...   네이버 웹툰   0.468846

[10 rows x 8 columns]

In [ ]:
get_recomm('트리거')

     writer illustrator    genre
62      고경빈         고경빈      판타지
1513   불만있냥        불만있냥  학원, 판타지
원하는 작품을 선택하세요. : 2


title writer  ... platform similarity
1621  정의는 죽지 않는다    크레용  ...    카카오웹툰   0.646432
1644        크라우드    만두비  ...    카카오웹툰   0.642245
1471          액괴    서강용  ...    카카오웹툰   0.640791
1640        가드패스   오손도손  ...    카카오웹툰   0.640001
1539      방구석 여포   키미하이  ...    카카오웹툰   0.639403
2288     정령의 펜던트     빅피  ...    카카오웹툰   0.639176
1611     이세계 전담반    손희준  ...    카카오웹툰   0.637391
2365   옥탑방 소드마스터    김의권  ...    카카오웹툰   0.636615
1510        개목걸이    김종욱  ...    카카오웹툰   0.636145
1954      일진의 크기     윤필  ...    카카오웹툰   0.636143

[10 rows x 8 columns]

In [ ]:
get_recomm('이태원 클라쓰')

title writer  ... platform similarity
2174  커피와 하루 the start    한상훈  ...    카카오웹툰   0.506494
895         데우스 엑스 마키나    꼬마비  ...   네이버 웹툰   0.500186
1963         아 지갑놓고나왔다  미역의효능  ...    카카오웹툰   0.487160
653                 회춘   기안84  ...   네이버 웹툰   0.486529
2156               흰둥이     윤필  ...    카카오웹툰   0.480994
757               오라존미   허5파6  ...   네이버 웹툰   0.480338
1910        벨트 위 벨트 아래    마영신  ...    카카오웹툰   0.479863
1435              사노라면    네이버  ...   네이버 웹툰   0.478531
1949            인천상륙작전    윤태호  ...    카카오웹툰   0.475568
4008             명동젠틀맨     직가  ...    레진코믹스   0.472401

[10 rows x 8 columns]

In [ ]:
get_recomm('이태원 클라쓰',w_author=1.5,w_genre=1.0)

title writer  ... platform similarity
658               링크보이     광진  ...   네이버 웹툰   0.346154
2174  커피와 하루 the start    한상훈  ...    카카오웹툰   0.179821
1963         아 지갑놓고나왔다  미역의효능  ...    카카오웹툰   0.168904
653                 회춘   기안84  ...   네이버 웹툰   0.168548
2156               흰둥이     윤필  ...    카카오웹툰   0.165423
1910        벨트 위 벨트 아래    마영신  ...    카카오웹툰   0.164784
1435              사노라면    네이버  ...   네이버 웹툰   0.164032
1949            인천상륙작전    윤태호  ...    카카오웹툰   0.162359
4008             명동젠틀맨     직가  ...    레진코믹스   0.160571
895         데우스 엑스 마키나    꼬마비  ...   네이버 웹툰   0.160547

[10 rows x 8 columns]

In [ ]:
get_recomm('이태원 클라쓰',w_author=1.0,w_genre=1.5)

title writer  ... platform similarity
2174  커피와 하루 the start    한상훈  ...    카카오웹툰   0.460730
895         데우스 엑스 마키나    꼬마비  ...   네이버 웹툰   0.452451
1963         아 지갑놓고나왔다  미역의효능  ...    카카오웹툰   0.447215
653                 회춘   기안84  ...   네이버 웹툰   0.446774
2156               흰둥이     윤필  ...    카카오웹툰   0.442904
1910        벨트 위 벨트 아래    마영신  ...    카카오웹툰   0.442113
1435              사노라면    네이버  ...   네이버 웹툰   0.441183
757               오라존미   허5파6  ...   네이버 웹툰   0.440511
1949            인천상륙작전    윤태호  ...    카카오웹툰   0.439111
4008             명동젠틀맨     직가  ...    레진코믹스   0.436897

[10 rows x 8 columns]

In [7]:
get_recomm('비흔','삼국지톡',w_author=1.0,w_genre=1.5)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:2898: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_loc(casted_key)


KeyError: ignored

In [8]:
idx = indices['비흔','삼국지톡']

/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:2898: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_loc(casted_key)


KeyError: ignored